In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
from learnMSA import msa_hmm
from matplotlib import pyplot as plt
import pandas as pd

## MSA HMM Interactive

1. Fit n models, keep the best and align
2. Compare to a reference
3. Visualize the HMM

Change the variables in the following cell to fit your needs.

In [7]:
# Your fasta file with unaligned sequences.

dataset = "ext_homfam_medium"
family = "B_lectin"
train_filename = f"../snakeMSA/data/{dataset}/train/{family}"
#train_filename = "test/data/egf.fasta"

# Reference file with aligned sequences that have matching IDs to (potentially a subset of) the 
# sequences in the train_file.
# Replace with empty string if no reference is available.
ref_filename = f"../snakeMSA/data/{dataset}/refs/{family}"
#ref_filename = "test/data/egf.ref"

# The number of independently trained models.
num_models = 10

## Training

In [ ]:
!mamba install famsa t-coffee mmseqs2 -y

In [3]:
#clustering based sequence weights
!mmseqs easy-cluster ../snakeMSA/data/{dataset}/train/{family} test_cluster tmp --cov-mode 1 --min-seq-id 0.6 -v 0

In [8]:
clustering = pd.read_csv('test_cluster_cluster.tsv', sep="\t", names=["representative", "sequence"])
cluster_counts = clustering.groupby("representative").size().to_frame("cluster_size")
clustering = clustering.merge(cluster_counts, how="left", on="representative")
clustering["weight"] = 1/clustering["cluster_size"]
clustering = clustering.set_index("sequence")

In [9]:
clustering

,representative,cluster_size,weight
sequence,,,
PQM36978.1/76-185,PQM36978.1/76-185,15,0.066667
XP_021826607.1/111-220,PQM36978.1/76-185,15,0.066667
VVA33926.1/97-206,PQM36978.1/76-185,15,0.066667
XP_008227125.1/90-199,PQM36978.1/76-185,15,0.066667
ONI13682.1/107-214,PQM36978.1/76-185,15,0.066667
...,...,...,...
PUZ74852.1/79-156,TVU08192.1/75-158,4,0.250000
PVH65907.1/79-156,TVU08192.1/75-158,4,0.250000
XP_015901406.1/87-168,XP_015901406.1/87-168,1,1.000000


In [10]:
clustering.loc[clustering.index == "1707266A/84-190"]

,representative,cluster_size,weight
sequence,,,
1707266A/84-190,AAS94120.1/82-193,294,0.003401


In [11]:
fasta_file = msa_hmm.fasta.Fasta(train_filename)
ids = fasta_file.seq_ids
for i in range(len(ids)):
    if "|" in ids[i]:
        pos = ids[i].rfind("|")
        if pos != -1:
            ids[i] = ids[i][pos+1:]
sequence_weights = np.array(clustering.loc[ids].weight, dtype=np.float32)
sequence_weights

array([0.5       , 1.        , 0.5       , ..., 0.33333334, 0.125     ,
       0.05882353], dtype=float32)

In [12]:
np.sum(sequence_weights)

1835.0001

In [ ]:
out_filename = "test/data/interactive.alignment.fasta"
config = msa_hmm.config.make_default(num_models)

config["min_surgery_seqs"] = 1e5

#maybe change alignment mode parameters
transitioners = config["transitioner"] if hasattr(config["transitioner"], '__iter__') else [config["transitioner"]]
#for trans in transitioners:
#        trans.prior.alpha_flank = 100
#        trans.prior.alpha_single = 100
#        trans.prior.alpha_global = 30
        #trans.prior.alpha_flank_compl = 0.1
        #trans.prior.alpha_single_compl = 0.1
        #trans.prior.alpha_global_compl = 0.1

alignment_model = msa_hmm.align.run_learnMSA(train_filename,
                                              out_filename,
                                              config, 
                                              sequence_weights=sequence_weights,
                                              verbose=True,
                                              align_insertions=True)
#msa_hmm.vis.print_and_plot(alignment_model, alignment_model.best_model)

Training of 10 models on file B_lectin
Configuration: 
{
num_models : 10
transitioner : ProfileHMMTransitioner(
 transition_init=
    {
    begin_to_match : DefaultEntry() , match_to_end : DefaultExit() , 
    match_to_match : DefaultMatchTransition(1) , match_to_insert : DefaultMatchTransition(-1) , 
    insert_to_match : Norm(0, 0.1) , insert_to_insert : Norm(-0.5, 0.1) , 
    match_to_delete : DefaultMatchTransition(-1) , delete_to_match : Norm(0, 0.1) , 
    delete_to_delete : Norm(-0.5, 0.1) , left_flank_loop : Norm(0, 0.1) , 
    left_flank_exit : Norm(-1, 0.1) , right_flank_loop : Norm(0, 0.1) , 
    right_flank_exit : Norm(-1, 0.1) , unannotated_segment_loop : Norm(0, 0.1) , 
    unannotated_segment_exit : Norm(-1, 0.1) , end_to_unannotated_segment : Norm(-9, 0.1) , 
    end_to_right_flank : Norm(0, 0.1) , end_to_terminal : Norm(0, 0.1)
    },
 flank_init=Const(0.0),
 prior=ProfileHMMTransitionPrior(match_comp=1, insert_comp=1, delete_comp=1, alpha_flank=7000, alpha_single=1000

In [ ]:
!id_list=$(sed -n '/^>/p' {ref_filename} | sed 's/^.//') ; export MAX_N_PID_4_TCOFFEE=10000000 ; t_coffee -other_pg seq_reformat -in test/data/interactive.alignment.fasta -action +extract_seq_list ${{id_list[@]}} +rm_gap > test/data/interactive.projection.fasta

In [ ]:
!t_coffee -other_pg aln_compare -al1 {ref_filename} -al2 test/data/interactive.projection.fasta -compare_mode sp

In [66]:
!head ../snakeMSA/famsa/scores/{dataset}/{family}

ext_homfam_medium B_lectin 97.5 97.5 96.3 96.3


In [67]:
!head ../snakeMSA/learnMSA/scores/{dataset}/{family}

ext_homfam_medium B_lectin 27.8 41.1 0.0 0.0


In [109]:
!head ../snakeMSA/data/{dataset}/refs/{family}

>P1_1jpc
DNILYSGETLSTGEFLNYGSFVFIMQEDCNLVLYDVDKPIWATNTGG-LSRSCFLSMQTDGNLVVYNPSN
KPIWASNTGGQNGNYVCILQKDRNVVIYGTDRWATGTHT-
>P1_1bwud
RNILTNDEGLYGGQSLDVNPYHLIMQEDCNLVLYDHSTAVWSSNTDIPGKKGCKAVLQSDGNFVVYDAEG
ASLWASHSVRGNGNYVLVLQEDGNVVIYRSDIWSTNTYR-
>P1_1npla
DNILYSGETLSPGEFLNNGRYVFIMQEDCNLVLYDVDKPIWATNTGG-LDRRCHLSMQSDGNLVVYSPRN
NPIWASNTGGENGNYVCVLQKDRNVVIYGTARWATGTNIH


In [110]:
!head -n 20 test/data/interactive.projection.fasta

>P1_1jpc
dnilysgetlstgeflnygsfvfimqedcnlvlydvdkpiwatntggls-
rscflsmqtdgnlvvynpsnkpiwasntggqngnyvcilqkdrnvviygt
drwatgtht...
>P1_1bwud
rniltndeglyggqsldvnpyhlimqedcnlvlydhstavwssntdipgk
kgckavlqsdgnfvvydaegaslwashsvrgngnyvlvlqedgnvviyrs
diwstntyr...
>P1_1npla
dnilysgetlspgeflnngryvfimqedcnlvlydvdkpiwatn-tggld
rrchlsmqsdgnlvvysprnnpiwasntggengnyvcvlqkdrnvviyg-
-tarwatgtnih


In [111]:
!head -n 20 ../snakeMSA/data/{dataset}/train/{family}

>P1_1jpc
DNILYSGETLSTGEFLNYGSFVFIMQEDCNLVLYDVDKPIWATNTGGLSRSCFLSMQTDGNLVVYNPSNK
PIWASNTGGQNGNYVCILQKDRNVVIYGTDRWATGTHT
>P1_1bwud
RNILTNDEGLYGGQSLDVNPYHLIMQEDCNLVLYDHSTAVWSSNTDIPGKKGCKAVLQSDGNFVVYDAEG
ASLWASHSVRGNGNYVLVLQEDGNVVIYRSDIWSTNTYR
>P1_1npla
DNILYSGETLSPGEFLNNGRYVFIMQEDCNLVLYDVDKPIWATNTGGLDRRCHLSMQSDGNLVVYSPRNN
PIWASNTGGENGNYVCVLQKDRNVVIYGTARWATGTNIH
>KVI10412.1/337-445
TVAWVANRETPIANKSGELTLNPDGVLVLRDSItNRIMWSSNATSTIQNPVARLLDTGNLMVvdgdDDSN
DPENYIWQSFDHPTDTFLPDLKFGRNLKKGVVTNFTSWK
>KVI10412.1/1035-1141
TVAWVANREIPIRNNSGELTLHSDGVLVLRDSTtNTIVWSTSSPGTTTGNPVARLSDSGNLVVvNDDNEP
ENYIWQSFDHPGDTVLPGMKFGRDLEKGIVTNVTSWK
>KVI10412.1/1779-1884
TVAWVANREIPIRNNSGELTLHSDGVLVLRDSTtNQVVWSSTSSETAENPVARLLDSGNLMVvDRDDGPE
NYIWQSFDYPGDTALAGVKVGRNLERGVVTNLTSWK
>KVI10412.1/2416-2523
TVVWVANRETPIRNKTGELTLHPDGVLELRDTAtDIIVWSTNTKGSAQNLVARLLDSGNLVVidnDDDNQ
